In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from dataset import VNAgriDataset

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

import pickle
import bisect

In [2]:
caphe_df = pd.read_csv("../../data/pre_caphe.csv")
luagao_df = pd.read_csv("../../data/pre_luagao.csv")
rauqua_df = pd.read_csv("../../data/pre_rauqua.csv")

In [3]:
nganh_caphe = pd.Series(["Cà phê"] * len(caphe_df), name="Ngành_hàng")
nganh_luagao = pd.Series(["Lua gao"] * len(luagao_df), name="Ngành_hàng")
nganh_rauqua = pd.Series(["Rau quả"] * len(rauqua_df), name="Ngành_hàng")

In [4]:
caphe_df = pd.concat([caphe_df, nganh_caphe], axis=1)
luagao_df = pd.concat([luagao_df, nganh_luagao], axis=1)
rauqua_df = pd.concat([rauqua_df, nganh_rauqua], axis=1)

In [5]:
pre_data = pd.concat([caphe_df, luagao_df, rauqua_df], axis=0, ignore_index=True)

In [6]:
pre_data.to_csv("../../data/pre_data.csv", index=False)

In [7]:
pre_data.shape

(48736, 9)

In [8]:
pre_data.head()

,Tên_mặt_hàng,Thị_trường,Loại_giá,Đơn_vị_tính,Loại_tiền,Nguồn,Ngày,Giá,Ngành_hàng
0,Cà phê Robusta nhân xô,Đắk Lắk,Thương lái thu mua,Vnđ/Kg,VNĐ,CTV địa phương,2025-05-09,128233.0,Cà phê
1,Cà phê Robusta nhân xô,Đắk Nông,Thương lái thu mua,Vnđ/Kg,VNĐ,CTV địa phương,2025-05-09,128350.0,Cà phê
2,Cà phê Robusta nhân xô,Gia Lai,Thương lái thu mua,Vnđ/Kg,VNĐ,CTV địa phương,2025-05-09,128233.0,Cà phê
3,Cà phê Robusta nhân xô,Kon Tum,Thương lái thu mua,Vnđ/Kg,VNĐ,CTV địa phương,2025-05-09,128200.0,Cà phê
4,Cà phê Robusta nhân xô,Lâm Đồng,Thương lái thu mua,Vnđ/Kg,VNĐ,CTV địa phương,2025-05-09,128000.0,Cà phê


# Kiểm tra cột tên mặt hàng

In [9]:
pre_data["Tên_mặt_hàng"].unique()[:5]

array(['Cà phê Robusta nhân xô', 'Cà phê Arabica nhân xô', 'CLC 4900',
       'Jasmine', 'OM 18'], dtype=object)

In [10]:
print(len(pre_data["Tên_mặt_hàng"].unique()))

394


## Xoá các mặt hàng có số lượng record ít

In [11]:
items = pre_data["Tên_mặt_hàng"].unique()
avg_n_record = 0.
n_items = len(items)

for item in items:
    n_item = len(pre_data[pre_data["Tên_mặt_hàng"] == item])
    avg_n_record += n_item/n_items

In [12]:
avg_n_record

123.69543147208128

In [13]:
pre_data = pre_data.groupby("Tên_mặt_hàng").filter(lambda x: len(x) >= avg_n_record)

In [14]:
len(pre_data["Tên_mặt_hàng"].unique())

117

# Kiểm tra cột thị trường

In [15]:
pre_data["Thị_trường"].unique()

array(['Đắk Lắk', 'Đắk Nông', 'Gia Lai', 'Kon Tum', 'Lâm Đồng',
       'Hồ Chí Minh', 'Cần Thơ', 'Sóc Trăng', 'Đồng Tháp', 'Kiên Giang',
       'An Giang', 'Tiền Giang', 'Trà Vinh', 'Hậu Giang', 'Thái Bình',
       'Hà Nội', 'Bến Tre', 'Cà Mau', 'Long An', 'Vĩnh Long', 'Bạc Liêu',
       'Sơn La'], dtype=object)

In [16]:
print(len(pre_data["Thị_trường"].unique()))

22


# Kiểm tra cột loại giá

In [17]:
pre_data["Loại_giá"].unique()

array(['Thương lái thu mua', 'Công ty thu mua', 'Đại lý thu mua',
       'Thu mua', 'Bán ra', 'Tại chợ', 'Bán lẻ', 'Khác',
       'Thu mua tại vườn', 'Bán buôn', 'Vựa thu mua'], dtype=object)

In [18]:
print(len(pre_data["Loại_giá"].unique()))

11


# Kiểm tra đơn vị tính

In [19]:
pre_data["Đơn_vị_tính"].unique()

array(['Vnđ/Kg', 'VNĐ/kg', 'VNĐ/Kg', 'Đồng/kg', 'VNĐ/quả', 'VNĐ/mớ',
       'VNĐ/Chục quả', 'VNĐ/củ', 'VNĐ/Quả'], dtype=object)

In [20]:
print(len(pre_data["Đơn_vị_tính"].unique()))

9


In [21]:
pre_data["Đơn_vị_tính"] = pre_data["Đơn_vị_tính"].apply(
    lambda x: "Đồng/kg" if x in [
        'Vnđ/Kg', 'VNĐ/kg', 'VNĐ/Kg', 'Đồng/kg', 'VNĐ/củ', 'VNĐ/Quả'] else x
)

In [22]:
pre_data["Đơn_vị_tính"].unique()

array(['Đồng/kg', 'VNĐ/quả', 'VNĐ/mớ', 'VNĐ/Chục quả'], dtype=object)

# Kiểm tra cột loại tiền

In [23]:
pre_data["Loại_tiền"].unique()

array(['VNĐ'], dtype=object)

In [24]:
del pre_data["Loại_tiền"]

In [25]:
pre_data.head()

,Tên_mặt_hàng,Thị_trường,Loại_giá,Đơn_vị_tính,Nguồn,Ngày,Giá,Ngành_hàng
0,Cà phê Robusta nhân xô,Đắk Lắk,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128233.0,Cà phê
1,Cà phê Robusta nhân xô,Đắk Nông,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128350.0,Cà phê
2,Cà phê Robusta nhân xô,Gia Lai,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128233.0,Cà phê
3,Cà phê Robusta nhân xô,Kon Tum,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128200.0,Cà phê
4,Cà phê Robusta nhân xô,Lâm Đồng,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128000.0,Cà phê


# Lấy các sản phẩm được cập nhật đến 2025

In [26]:
pre_data["Ngày"] = pd.to_datetime(pre_data["Ngày"])

In [27]:
items_2025 = pre_data[pre_data["Ngày"].dt.year == 2025]["Tên_mặt_hàng"].unique()

In [28]:
len(items_2025)

74

In [29]:
filtered_data = pre_data[pre_data["Tên_mặt_hàng"].isin(items_2025)]

In [30]:
len(filtered_data)

28726

# Lập metadata trước khi được chuẩn hoá thời gian và Label Encoder

# Chuẩn hoá mốc thời gian

## Thêm 1 cột ngày được chuyển sang kiểu int cho dễ

In [31]:
filtered_data.columns

Index(['Tên_mặt_hàng', 'Thị_trường', 'Loại_giá', 'Đơn_vị_tính', 'Nguồn',
       'Ngày', 'Giá', 'Ngành_hàng'],
      dtype='object')

In [32]:
filtered_data.head()

,Tên_mặt_hàng,Thị_trường,Loại_giá,Đơn_vị_tính,Nguồn,Ngày,Giá,Ngành_hàng
0,Cà phê Robusta nhân xô,Đắk Lắk,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128233.0,Cà phê
1,Cà phê Robusta nhân xô,Đắk Nông,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128350.0,Cà phê
2,Cà phê Robusta nhân xô,Gia Lai,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128233.0,Cà phê
3,Cà phê Robusta nhân xô,Kon Tum,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128200.0,Cà phê
4,Cà phê Robusta nhân xô,Lâm Đồng,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128000.0,Cà phê


In [33]:
items = filtered_data["Tên_mặt_hàng"].unique()

In [34]:
items[:5]

array(['Cà phê Robusta nhân xô', 'Jasmine', 'OM 18', 'OM 5451',
       'Đài Thơm 8'], dtype=object)

In [35]:
from datetime import datetime

start_date = filtered_data["Ngày"].min()
end_date = filtered_data["Ngày"].max()

In [36]:
filtered_data["Ngày(int)"] = (filtered_data["Ngày"] - start_date).dt.days

C:\Users\DELL\AppData\Local\Temp\ipykernel_5992\827847078.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["Ngày(int)"] = (filtered_data["Ngày"] - start_date).dt.days


In [37]:
filtered_data.head()

,Tên_mặt_hàng,Thị_trường,Loại_giá,Đơn_vị_tính,Nguồn,Ngày,Giá,Ngành_hàng,Ngày(int)
0,Cà phê Robusta nhân xô,Đắk Lắk,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128233.0,Cà phê,1955
1,Cà phê Robusta nhân xô,Đắk Nông,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128350.0,Cà phê,1955
2,Cà phê Robusta nhân xô,Gia Lai,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128233.0,Cà phê,1955
3,Cà phê Robusta nhân xô,Kon Tum,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128200.0,Cà phê,1955
4,Cà phê Robusta nhân xô,Lâm Đồng,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128000.0,Cà phê,1955


In [38]:
weekly_index = pd.date_range(
    start=start_date,
    end=end_date,
    freq="W"
)

In [39]:
weekly_index = (weekly_index - start_date).days

## Sử dụng nội suy tuyến tinh để tính các điểm dữ liệu

In [ ]:
import pandas as pd
import numpy as np
import bisect

# Chuẩn bị danh sách để chứa kết quả
interpolated_rows = []

for item in items:
    item_df = filtered_data[filtered_data["Tên_mặt_hàng"] == item]

    # Lấy giá trị ngày dưới dạng int (số ngày từ start_date)
    date_int = item_df["Ngày(int)"]
    
    min_item_date = np.min(date_int)
    max_item_date = np.max(date_int)

    # Tìm chỉ số trong weekly_index tương ứng
    min_week_range = bisect.bisect_left(weekly_index, min_item_date)
    max_week_range = bisect.bisect_right(weekly_index, max_item_date)

    # Các mốc tuần cần lấy
    week_item_range = weekly_index[min_week_range: max_week_range]

    # Nội suy giá trị
    item_interp = np.interp(week_item_range, date_int, item_df["Giá"])

    # Với mỗi tuần, tạo một dòng mới giữ lại các thuộc tính khác (dùng mode hoặc first)
    for week_day, price in zip(week_item_range, item_interp):
        row = {
            "Tên_mặt_hàng": item,
            "Ngày(int)": week_day,
            "Giá": price
        }
        # Thêm các cột khác nếu có
        for col in ['Thị_trường', 'Loại_giá', 'Đơn_vị_tính', 'Nguồn', 'Ngành_hàng']:
            if col in item_df.columns:
                row[col] = item_df[col].iloc[0]
        interpolated_rows.append(row)

# Gộp lại thành DataFrame
interpolated_df = pd.DataFrame(interpolated_rows)

# Nếu muốn, có thể chuyển lại "Ngày(int)" → "Ngày"
interpolated_df["Ngày"] = pd.to_datetime(start_date) + pd.to_timedelta(interpolated_df["Ngày(int)"], unit="D")


In [49]:
len(interpolated_df["Ngày(int)"].unique())

281